In [ ]:
def count_eye_blinks_v2(eeg_epochs, sfreq, channel_names, frontal_channels=['Fz', 'F4']):
    """
    Returns a list of ratio (peak_amp/variance) for each subepoch in all epochs,
    using time-domain peak amplitude.
    """
    frontal_idxs = [i for i, ch in enumerate(channel_names) if ch in frontal_channels]
    if not frontal_idxs:
        raise ValueError("No frontal channels found in data.")

    ratios = []
    peaks = []
    variances = []

    for epoch in eeg_epochs:
        # Each epoch is length 1279 instead of 1280, so zero-pad
        if epoch.shape[1] == 1279:
            epoch = np.pad(epoch, ((0, 0), (0, 1)), mode='constant')
            
        n_samples = epoch.shape[1]
        n_subepochs = 10
        subepoch_len = 128

        # Only process if epoch is long enough
        if n_samples < n_subepochs * subepoch_len:
            continue

        for i in range(n_subepochs):
            start = i * subepoch_len
            end = start + subepoch_len
            sub_epoch = epoch[:, start:end]

            # Data from 2 frontal electrodes
            frontal_avg = np.mean(sub_epoch[frontal_idxs, :], axis=0)

            # Get the time-domain peak amplitude per subepoch
            peak_amp = np.max((frontal_avg))
            peaks.append(peak_amp)

            # Get the variance for the subepoch
            variance = np.std(frontal_avg)
            variances.append(variance)

            # Calculate the ratio between the peak amplitude and variance of the subepoch
            ratio = peak_amp / variance if variance != 0 else np.nan
            ratios.append(ratio)

    return ratios, peaks, variances



In [ ]:
# Get one epoch (e.g., first participant, first stimulus, first epoch)
single_epoch = epochs_organized[0][0][0]  # shape: (n_channels, n_samples)

# Call the function for just this epoch
ratios, peaks, variances = count_eye_blinks_v2(
    eeg_epochs=[single_epoch],  # Pass as a list
    sfreq=eeg_fs,
    channel_names=ch_names
)

print(ratios)  # This will be a list of ratios for each subepoch in that epoch
print(peaks)
print(variances)